In [2]:
import numpy as np

def policy_evaluation_gbike(policy, Lamda, lamda, r, t, gam):
# Implement policy evaluation for gbike problem
# Returns state value V for the given policy
    n_states = (21)**2
    n_actions = 11
    T = np.zeros((n_states, n_states, n_actions))
    R = np.zeros((n_states, n_actions))
    P = np.zeros((n_states, n_actions, n_states))
    V = np.zeros((21, 21))

    # Calculate transition probabilities and rewards for all states and actions
    for i in range(21):
        for j in range(21):
            state = i*21 + j
            for action in range(n_actions):
                na1 = min(i+action-5, 20) # number of available bikes at location 1 after action
                na2 = min(j-action+5, 20) # number of available bikes at location 2 after action
                for r1 in range(21):
                    for r2 in range(21):
                        p = poisson_prob(r1, Lamda[0]) * poisson_prob(r2, Lamda[1]) # probability of rental requests
                        rent1 = min(na1, r1) # number of bikes rented at location 1
                        rent2 = min(na2, r2) # number of bikes rented at location 2
                        reward = (rent1 + rent2) * r # rental reward
                        nr1 = na1 - rent1 + min(lamda[0], 20-na1+rent1) # number of available bikes at location 1 after returns
                        nr2 = na2 - rent2 + min(lamda[1], 20-na2+rent2) # number of available bikes at location 2 after returns
                        ns = nr1*21 + nr2 # next state
                        R[state, action] += p * reward # reward for current state and action
                        T[state, ns, action] += p # transition probability for current state, action, and next state

    # Perform policy evaluation
    while True:
        delta = 0
        for i in range(21):
            for j in range(21):
                state = i*21 + j
                v = 0
                for action in range(n_actions):
                    v_a = 0
                    for ns in range(n_states):
                        v_a += T[state, ns, action] * (R[state, action] + gam * V[ns//21, ns%21])
                    v += policy[i, j, action] * v_a
                delta = max(delta, abs(v - V[i, j]))
                V[i, j] = v
        if delta < 1e-4:
            break
    return V


In [3]:
def policy_improvement_gbike(V, policy, Lamda, lamda, r, t, gam):
# Implement policy improvement for gbike problem
# Returns new policy and a boolean indicating if the policy has stabilized

    n_states = (21)**2
    n_actions = 11
    new_policy = np.zeros((21, 21, n_actions))
    P = np.zeros((n_states, n_actions, n_states))
    R = np.zeros((n_states, n_actions))
    policystable = True
    for i in range(21):
        for j in range(21):
            state = i*21 + j
            for action in range(n_actions):
                na1 = min(i+action-5, 20) # number of available bikes at location 1 after action
                na2 = min(j-action+5, 20) # number of available bikes at location 2 after action
                for r1 in range(21):
                    for r2 in range(21):
                        p = poisson_prob(r1, Lamda[0]) * poisson_prob(r2, Lamda[1]) # probability of rental requests
                        rent1 = min(na1, r1) # number of bikes rented at location 1
                        rent2 = min(na2, r2) # number of bikes rented at location 2
                        reward = (rent1 + rent2) * r # rental reward
                        nr1 = na1 - rent1 + min(lamda[0], 20-na1+rent1) # number of available bikes at location 1 after returns
                        nr2 = na2 - rent2 + min(lamda[1], 20-na2+rent2) # number of available bikes at location 2 after returns
                        ns = nr1*21 + nr2 # next state
                        R[state, action] += p * reward # reward for current state and action
                        P[state, action, ns] += p # transition probability for current state, action, and next state

    # Perform policy improvement
    for i in range(21):
        for j in range(21):
            state = i*21 + j
            q = np.zeros(n_actions)
            for action in range(n_actions):
                for ns in range(n_states):
                    q[action] += P[state, action, ns] * (R[state, action] + gam * V[ns//21, ns%21])
            best_action = np.argmax(q)
            new_policy[i, j, best_action] = 1
            if not np.array_equal(new_policy[i, j, :], policy[i, j, :]):
                policystable = False
    return new_policy, policystable

In [4]:
def poisson_prob(n, lam):
# Calculate the probability of n occurrences given the rate parameter lam using Poisson distribution
    return lam**n * np.exp(-lam) / np.math.factorial(n)